## Data ETL Notebook

**Layer**: Silver

**Domain**: Risk-free

**Action**: Clean NZDM Amounts - Select current only, clean the coupon column to be numeric and convert date first issued to date type

In [0]:
%sql
-- Read current records from Bronze and write to Silver table (overwrite current version)
CREATE OR REPLACE TABLE workspace.riskfree_silver.001_nzdm_govtbonds_onissue_cleaned AS
SELECT 
    TO_DATE(CONCAT('15/', date_first_issued), 'dd/M/yyyy')  -- handles '4/2024', '10/2012'
    AS date_first_issued,

    loan_prefix,
    
    -- Clean and convert coupon to numeric
    CASE 
        WHEN coupon LIKE '%\%%' THEN CAST(REGEXP_REPLACE(coupon, '%', '') AS DOUBLE) / 100
        ELSE CAST(coupon AS DOUBLE)
    END AS coupon,

    maturity,
    total_amt_outstanding_incl_rb_eqc_sresl_m,
    rbnz_m,
    eqc_m,
    sresl_m,
    market_bonds_m,
    publish_date,
    as_of_date,
    source_file_name,
    ingestion_timestamp
FROM workspace.riskfree_bronze.nzdm_govtbonds_onissue
WHERE is_current = true;

-- Confirmation message
SELECT '✅ Silver table created: workspace.riskfree_silver.001_nzdm_govtbonds_onissue_cleaned' AS message;